In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, confusion_matrix, classification_report, matthews_corrcoef)
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# Train and test the model
def train_model(X_train, y_train, X_test, y_test, feature_names, is_pca = False, grid_search=False):
    # Initialize and fit the model
    start_model = RandomForestClassifier(
        verbose=1,
        n_jobs=-1,
        n_estimators=200,
        min_samples_leaf=2, 
        min_samples_split=10,
        max_depth=10,
        max_features=None,
        random_state=42,
        class_weight='balanced'
    )

    if grid_search:
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': ['sqrt', 'log2', None],
        }

        # Searching for the best tree parameters
        grid_search = GridSearchCV(
            start_model,
            param_grid,
            n_jobs=-1
        )

        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_

        print("best_params_ ----> Random Forest:", best_params)
        print("best_rf_model: ", best_model)
    else:
        best_model = start_model
        best_model.fit(X_train, y_train)

    # Test predictions
    y_pred = best_model.predict(X_test)

    # Evaluate model
    metrics_rf = calculate_performance_metrics(y_test, y_pred)
    print_performance_metrics(metrics_rf)
    if not is_pca:
        feature_importance(best_model, X_train, feature_names)
    return best_model

# Useful values for classification
def calculate_performance_metrics(y_test, y_pred):
    metrics = {}
    metrics['accuracy'] = accuracy_score(y_test, y_pred)
    metrics['precision'] = precision_score(y_test, y_pred, average='weighted')
    metrics['recall'] = recall_score(y_test, y_pred, average='weighted')
    metrics['f1_score'] = f1_score(y_test, y_pred, average='weighted')
    metrics['confusion_matrix'] = confusion_matrix(y_test, y_pred)
    metrics['mcc'] = matthews_corrcoef(y_test, y_pred)
    metrics['classification_report'] = classification_report(y_test, y_pred)
    
    return metrics

# Prints all performance metrics
def print_performance_metrics(metrics):
    print("Accuracy:", metrics.get('accuracy', "Not computed"))
    print("Precision:", metrics.get('precision', "Not computed"))
    print("Recall:", metrics.get('recall', "Not computed"))
    print("F1 Score:", metrics.get('f1_score', "Not computed"))
    print("Confusion Matrix:\n", metrics.get('confusion_matrix', "Not computed"))
    print("Matthews Correlation Coefficient (MCC):", metrics.get('mcc', "Not computed"))
    print("Classification Report:\n", metrics.get('classification_report', "Not computed"))

# Determine the feature importance in the model
def feature_importance(model, X, feature_names, should_print=True):
    feature_importances = model.feature_importances_
    feature_importances_list = [(feature_names[j], importance) for j, importance in enumerate(feature_importances)]
    feature_importances_list.sort(key=lambda x: x[1], reverse=True)

    if should_print:
        for feature, importance in feature_importances_list:
            print(f"{feature}: {importance}")
    return feature_importances_list

In [9]:
# Load the data
def process_data(): 
    data_name = ''
    processed_data = pd.read_csv("./data/final.csv")
    processed_data = processed_data.drop(columns=["date"])


    # Separate data
    target_name = 'weather_code'
    X = processed_data.drop(columns=[target_name]).values
    y = processed_data[target_name].values

    # feature_names = processed_data.columns[:-1].tolist()
    feature_names = processed_data.drop(columns=[target_name]).columns.tolist()
    return X,y,feature_names
    

# Trains the model
def create_model():
    X,y,feature_names = process_data()

    # Get test and train
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print('Read the data')
    # Run the model
    model = train_model(X_train, y_train, X_test, y_test, feature_names)

    return model

In [10]:
# Runs the model
def run_model_training():
    # Train the model
    model = create_model()
    return model

model = run_model_training()

Read the data


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    0.2s


Accuracy: 0.9131205673758865
Precision: 0.9237610045960576
Recall: 0.9131205673758865
F1 Score: 0.9133527186854629
Confusion Matrix:
 [[ 34   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0  38   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0  34   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0 240   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0  44   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0  26   5   0   0   1   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  12   1   0   1   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   2   2   0   4   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   3   0   0  11   2   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   6  13   2   0   0   0   0   2   0]
 [  0   0   0   0   0   0   0   0   0   0

[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    1.1s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 200 out of 200 | elapsed:    0.0s finished
/opt/anaconda3/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, 

In [11]:
def remove_features():
    X,y,feature_names = process_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    importance_list = feature_importance(model, X_train, feature_names, False)
    features_to_remove = []
    total_importance = 0 
    for feature, importance in importance_list:  
        if total_importance >= 0.95:
            features_to_remove.append(feature)
        total_importance += importance
            
    
    print(f"Number of features to remove {len(features_to_remove)}")
    print(features_to_remove)
    features_to_remove.append("date")
    return features_to_remove

def run_model_reduced_data():
    features_to_remove = remove_features()
    
    processed_data = pd.read_csv("./data/final.csv")
    processed_data = processed_data.drop(columns=features_to_remove)
    
    
    # Separate data
    target_name = 'weather_code'
    X = processed_data.drop(columns=[target_name]).values
    y = processed_data[target_name].values
    
    # feature_names = processed_data.columns[:-1].tolist()
    feature_names = processed_data.drop(columns=[target_name]).columns.tolist()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    print('Read the data')
    new_model = train_model(X_train, y_train, X_test, y_test, feature_names)

run_model_reduced_data()


Number of features to remove 24
['uv_index_max', 'wind_gusts_10m_max', 'surface_pressure_mean', 'wind_speed_10m_mean', 'visibility_max', 'sunshine_duration', 'year_cos', 'et0_fao_evapotranspiration_sum', 'daylight_duration', 'temperature_2m_mean', 'cape_min', 'relative_humidity_2m_mean', 'relative_humidity_2m_max', 'wet_bulb_temperature_2m_mean', 'wind_speed_10m_min', 'year', 'day_of_week', 'relative_humidity_2m_min', 'visibility_mean', 'uv_index_clear_sky_max', 'month_sin', 'month', 'month_cos', 'showers_sum']
Read the data


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    0.1s


Accuracy: 0.9166666666666666
Precision: 0.9279580856873649
Recall: 0.9166666666666666
F1 Score: 0.9174402048657891
Confusion Matrix:
 [[ 34   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0]
 [  0  38   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0]
 [  0   0  34   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0]
 [  0   0   0 240   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0]
 [  0   0   0   0  44   0   0   0   0   0   0   0   0   0   0   0   0   0
    0]
 [  0   0   0   0   0  26   5   0   0   1   0   0   0   0   0   0   0   0
    0]
 [  0   0   0   0   0   0  12   1   0   1   0   0   0   0   0   0   0   0
    0]
 [  0   0   0   0   0   0   2   2   0   4   0   0   0   0   0   0   0   0
    0]
 [  0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0
    0]
 [  0   0   0   0   0   0   3   0   0  11   2   0   0   0   0   0   0   0
    0]
 [  0   0   0   0   0   0   0   0   0   6  14   1   0   

[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    0.7s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 200 out of 200 | elapsed:    0.0s finished
/opt/anaconda3/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [12]:

def pca(): 
    X,y,feature_names = process_data()
    # 1) Standardize features (very important for PCA if features have different scales)
    X_std = StandardScaler().fit_transform(X)
    
    # 2) Fit PCA (keep all components to see the full curve)
    pca = PCA(n_components=None, random_state=0)
    X_pca = pca.fit_transform(X_std)
    
    # 3) Explained variance
    evr = pca.explained_variance_ratio_             # per-component
    cev = np.cumsum(evr)                            # cumulative
    
    # 4) Choose a target variance (e.g., 95%)
    target = 0.95
    k = np.argmax(cev >= target) + 1                # smallest k reaching target
    
    print(f"Components to reach {target*100:.1f}% variance: {k}")
    
    loadings = pca.components_
    
    # contributions of each feature across first k PCs
    feature_contrib = np.sum((loadings[:k,:]**2).T * evr[:k], axis=1)
    
    # Normalize to 1
    feature_contrib = feature_contrib / feature_contrib.sum()
    
    #realate variance the feature contribution
    feature_importance = pd.Series(feature_contrib, index=feature_names)
    # sort descending
    feature_importance_sorted = feature_importance.sort_values(ascending=False)
    
    for feature, importance in feature_importance_sorted.items():
        print(f"{feature}: {importance:.4f}")

pca()
    



Components to reach 95.0% variance: 19
wind_direction_10m_dominant: 0.0172
winddirection_10m_dominant: 0.0172
day_of_week: 0.0172
day_of_month: 0.0172
apparent_temperature_mean: 0.0171
dew_point_2m_mean: 0.0171
wet_bulb_temperature_2m_mean: 0.0171
temperature_2m_mean: 0.0171
pressure_msl_mean: 0.0171
surface_pressure_mean: 0.0171
rain_sum: 0.0171
precipitation_sum: 0.0170
snowfall_water_equivalent_sum: 0.0170
snowfall_sum: 0.0170
apparent_temperature_max: 0.0169
apparent_temperature_min: 0.0169
wet_bulb_temperature_2m_min: 0.0169
temperature_2m_max: 0.0169
temperature_2m_min: 0.0169
wet_bulb_temperature_2m_max: 0.0169
year: 0.0168
dew_point_2m_max: 0.0168
dew_point_2m_min: 0.0168
cape_min: 0.0168
relative_humidity_2m_mean: 0.0166
cape_mean: 0.0166
wind_gusts_10m_mean: 0.0165
cape_max: 0.0165
et0_fao_evapotranspiration_sum: 0.0165
et0_fao_evapotranspiration: 0.0165
wind_speed_10m_mean: 0.0165
day_of_year: 0.0164
pressure_msl_max: 0.0164
month: 0.0164
surface_pressure_max: 0.0164
visibil